In [2]:
import torch

In [3]:
import numpy as np


In [5]:
a = torch.load('experiments/dynosplatam_davis/splatam_horsejump-high/splatam_horsejump-high_0_kNN_200_200_200_-1_32_False_0.5_20_20_5_0.001_True_True_True_True_True_2000_16_16_128_16_0.1_True_False_False_True_True_2_1_0_1_20_240_455_0.0_0.0_0_False_False_True_False_True_0.1_3_aniso_deb_l2_emb_r3/eval/debug.pth')

In [6]:
a.keys()


dict_keys(['min_idx', 'gs_traj_2D', 'gs_traj_3D', 'gt_traj_2D', 'occluded', 'pred_visibility'])

In [15]:
pred_visibility = a['pred_visibility']
comparison_visibility = pred_visibility.mean(dim=0).unsqueeze(1)

In [56]:
gs_traj_2D = a['gs_traj_2D']
comparison = gs_traj_2D.mean(dim=0).unsqueeze(1)
# comparison = a['gt_traj_2D'].unsqueeze(1)
# comparison[:, :, :, 0] *= 455
# comparison[:, :, :, 1] *= 240

In [53]:
print(comparison.shape, gs_traj_2D.shape)
print(comparison_visibility.shape, pred_visibility.shape)
print(pred_visibility.mean(dim=0).unsqueeze(0).shape)
print(gs_traj_2D.mean(dim=0).unsqueeze(0).shape)


torch.Size([25, 1, 50, 2]) torch.Size([10, 25, 50, 2])
torch.Size([25, 1, 50]) torch.Size([10, 25, 50])
torch.Size([1, 25, 50])
torch.Size([1, 25, 50, 2])


In [54]:
gs_traj_2D_perm = gs_traj_2D.permute(1, 0, 2, 3)
pred_visibility_perm = pred_visibility.permute(1, 0, 2)

In [63]:
dists = list()
vis_dists = list()
vars = list()
for p in range(gs_traj_2D_perm.shape[0]):
    d = torch.cdist(gs_traj_2D_perm[p].permute(1, 0, 2), comparison[p].permute(1, 0, 2))  
    v = pred_visibility_perm[p].permute(1, 0) -  comparison_visibility[p].permute(1, 0)
    dists.append(d)
    vis_dists.append(v)
    vars.append(torch.var(d, dim=0).squeeze())

vars = torch.stack(vars)
dists = torch.stack(dists)
vis_dists = torch.stack(vis_dists)
print(vars.shape,dists.shape, vis_dists.shape )
print(a['gt_traj_2D'].shape)
print(gs_traj_2D[a['min_idx'], torch.arange(gs_traj_2D.shape[1]), 0, :])


torch.Size([25, 10]) torch.Size([25, 50, 10, 1]) torch.Size([25, 50, 10])
torch.Size([10, 25, 50, 2])
tensor([[ 57.6701, 116.8360],
        [ 76.1047, 137.1631],
        [ 92.4431, 109.1113],
        [182.2782,  91.2516],
        [160.5210,  77.2249],
        [100.1818, 104.8924],
        [ 96.1117, 107.5409],
        [ 99.4224, 114.4367],
        [110.6395, 115.2574],
        [103.9640, 108.2380],
        [122.5278,  59.8487],
        [127.0003,  56.6334],
        [125.3885,  61.6155],
        [117.5775,  59.8784],
        [117.8646,  54.6548],
        [  0.4416,   1.7195],
        [126.9302, 110.4520],
        [122.4352, 114.1768],
        [116.5153, 123.3462],
        [  0.4416,   1.7195],
        [106.1834,  88.7647],
        [128.4322, 108.0312],
        [120.6127,  73.5627],
        [122.6227,  62.1555],
        [121.4648,  99.9884]])


In [2]:

from utils.slam_external import build_rotation
from datasets.gradslam_datasets.geometryutils import relative_transformation

def evaluate_ate(gt_traj, est_traj):
    """
    Input : 
        gt_traj: list of 4x4 matrices 
        est_traj: list of 4x4 matrices
        len(gt_traj) == len(est_traj)
    """
    gt_traj_pts = [gt_traj[idx][:3,3] for idx in range(len(gt_traj))]
    est_traj_pts = [est_traj[idx][:3,3] for idx in range(len(est_traj))]

    gt_traj_pts  = torch.stack(gt_traj_pts).detach().cpu().numpy().T
    est_traj_pts = torch.stack(est_traj_pts).detach().cpu().numpy().T

    _, _, trans_error = align(gt_traj_pts, est_traj_pts)

    avg_trans_error = trans_error.mean()

    return avg_trans_error


def align(model, data):
    """Align two trajectories using the method of Horn (closed-form).

    Args:
        model -- first trajectory (3xn)
        data -- second trajectory (3xn)

    Returns:
        rot -- rotation matrix (3x3)
        trans -- translation vector (3x1)
        trans_error -- translational error per point (1xn)

    """
    np.set_printoptions(precision=3, suppress=True)
    model_zerocentered = model - model.mean(1).reshape((3,-1))
    data_zerocentered = data - data.mean(1).reshape((3,-1))

    W = np.zeros((3, 3))
    for column in range(model.shape[1]):
        W += np.outer(model_zerocentered[:,
                         column], data_zerocentered[:, column])
    U, d, Vh = np.linalg.linalg.svd(W.transpose())
    S = np.matrix(np.identity(3))
    if (np.linalg.det(U) * np.linalg.det(Vh) < 0):
        S[2, 2] = -1
    rot = U*S*Vh
    trans = data.mean(1).reshape((3,-1)) - rot * model.mean(1).reshape((3,-1))

    model_aligned = rot * model + trans
    alignment_error = model_aligned - data

    trans_error = np.sqrt(np.sum(np.multiply(
        alignment_error, alignment_error), 0)).A[0]

    return rot, trans, trans_error


In [2]:
params_path = "experiments/dynosplatam_iphone/splatam_paper-windmill/splatam_paper-windmill_0_kNN_200_200_200_20_20_5_0.001_False_False_True_True_True_16_16_128_16_0.1_True_2_0_20_0.5_0.5_False_False_False_0.1_3_aniso_deb_l2_emb_aligned_depth_anything_colmap_refined_segment_bug_rem_factor_False_bug_rem_hw_deb"
config_path = "configs/iphone/dynosplatam_iphone.py"

params_path = "experiments/dynosplatam_davis/splatam_scooter-black/splatam_scooter-black_0_kNN_200_200_200_-1_32_False_0.5_20_20_5_0.001_True_True_True_True_True_2000_16_16_128_16_0.1_True_False_False_True_True_2_1_0_1_20_240_455_0.0_0.0_0_False_False_True_False_True_0.1_3_aniso_deb_l2_emb_r3"
config_path = "configs/davis/dynosplatam_davis.py"

params_path = "experiments/dynosplatam_jono/splatam_boxes/ims/27_0_kNN_500_1000_0_-1_32_False_True_False_0.5_20_20_5_0.001_False_False_True_True_True_2000_16_16_128_16_0.1_True_False_False_True_True_2_1_0_1_20_360_640_0.0_0.0_transformed"
config_path = "configs/jono_data/dynosplatam_jono_data_recent.py"


In [13]:
from utils.get_data import load_scene_data
import os
from importlib.machinery import SourceFileLoader
from utils.eval_traj_restructured import TrajEvaluator
import torch


params, _, _,first_frame_w2c = load_scene_data(None, params_path, device="cuda:0")

seq_experiment = SourceFileLoader(
        os.path.basename(config_path), config_path
        # 'config.py', os.path.join(p, 'config.py')
        ).load_module()
dev = "cuda:0"

evaluator = TrajEvaluator(
    seq_experiment.config,
    results_dir=params_path + '/eval',
    vis_trajs=False, # seq_experiment.config['viz']['vis_tracked'])
    best_x=1,
    traj_len=0,
    get_gauss_wise3D_track=True,
    get_from3D=False,
    vis_trajs_best_x=False,
    stereo=False,
    queries_first_t=True,
    vis_thresh=0.5,
    vis_thresh_start=0.5,
    primary_device=dev)


In [10]:
evaluator.vis_flow()

KeyboardInterrupt: 

In [7]:
def eval_cam_traj(params):
    gt_w2c_list = params['gt_w2c_all_frames']
    num_frames = params['cam_unnorm_rots'].shape[-1]
    latest_est_w2c = params['w2c']
    latest_est_w2c_list = []
    latest_est_w2c_list.append(latest_est_w2c)
    valid_gt_w2c_list = []
    valid_gt_w2c_list.append(gt_w2c_list[0])
    for idx in range(1, num_frames):
        # Check if gt pose is not nan for this time step
        if torch.isnan(gt_w2c_list[idx]).sum() > 0:
            continue
        interm_cam_rot = torch.nn.functional.normalize(params['cam_unnorm_rots'][..., idx].detach())
        interm_cam_trans = params['cam_trans'][..., idx].detach()
        intermrel_w2c = torch.eye(4).cuda().float()
        intermrel_w2c[:3, :3] = build_rotation(interm_cam_rot)
        intermrel_w2c[:3, 3] = interm_cam_trans
        latest_est_w2c = intermrel_w2c
        latest_est_w2c_list.append(latest_est_w2c)
        valid_gt_w2c_list.append(gt_w2c_list[idx])
    gt_w2c_list = valid_gt_w2c_list
    # Calculate ATE RMSE
    ate_rmse = evaluate_ate(gt_w2c_list, latest_est_w2c_list)
    print("Final Average ATE RMSE: {:.2f} cm".format(ate_rmse*100))

In [10]:
import torch
import numpy as np
eval_cam_traj(params)

Final Average ATE RMSE: 10.49 cm
